In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import os
import tqdm
import requests

from multiprocessing import Pool
import time
import itertools

import query_methods

In [3]:
import datetime

In [4]:
datetime.datetime.isoformat(datetime.datetime.now())[:-10]

'2020-03-25T00:52'

In [29]:
d = pd.read_pickle(os.path.join(os.getcwd(), "2019-12-13T00:50"))

In [21]:
match_id = 5117355338

result = requests.get(f"https://api.opendota.com/api/matches/{match_id}").json()

In [5]:
heroes = pd.DataFrame(requests.get(f"https://api.opendota.com/api/heroes").json())

In [39]:
output = []

for n in np.arange(100,500, 200):
    
    request = requests.get("https://api.opendota.com/api/proMatches?less_than_match_id={}7355338".format(n)).json()
    
    output.append(request)

In [41]:
pd.DataFrame(output)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,"{'match_id': 1007336904, 'duration': 2048, 'st...","{'match_id': 1007273774, 'duration': 2880, 'st...","{'match_id': 1007268164, 'duration': 1732, 'st...","{'match_id': 1007100692, 'duration': 1521, 'st...","{'match_id': 1007046405, 'duration': 2549, 'st...","{'match_id': 1007038410, 'duration': 2253, 'st...","{'match_id': 1006991368, 'duration': 1737, 'st...","{'match_id': 1006936170, 'duration': 2549, 'st...","{'match_id': 1006920184, 'duration': 3639, 'st...","{'match_id': 1006891573, 'duration': 2006, 'st...",...,"{'match_id': 1000416244, 'duration': 1838, 'st...","{'match_id': 1000392466, 'duration': 1457, 'st...","{'match_id': 1000352316, 'duration': 1224, 'st...","{'match_id': 1000330160, 'duration': 1935, 'st...","{'match_id': 1000318480, 'duration': 2660, 'st...","{'match_id': 1000309532, 'duration': 1429, 'st...","{'match_id': 1000308686, 'duration': 3936, 'st...","{'match_id': 1000290559, 'duration': 925, 'sta...","{'match_id': 1000226003, 'duration': 1442, 'st...","{'match_id': 1000191154, 'duration': 2496, 'st..."
1,"{'match_id': 3007293183, 'duration': 1341, 'st...","{'match_id': 3006969246, 'duration': 2109, 'st...","{'match_id': 3006919763, 'duration': 1385, 'st...","{'match_id': 3006862938, 'duration': 4305, 'st...","{'match_id': 3006807982, 'duration': 1864, 'st...","{'match_id': 3006738150, 'duration': 2439, 'st...","{'match_id': 3006699582, 'duration': 1227, 'st...","{'match_id': 3006479180, 'duration': 2123, 'st...","{'match_id': 3006373987, 'duration': 2188, 'st...","{'match_id': 3006260031, 'duration': 2667, 'st...",...,"{'match_id': 3000119731, 'duration': 3555, 'st...","{'match_id': 3000021292, 'duration': 2136, 'st...","{'match_id': 3000020100, 'duration': 5276, 'st...","{'match_id': 2999919744, 'duration': 2565, 'st...","{'match_id': 2999907435, 'duration': 2346, 'st...","{'match_id': 2999737824, 'duration': 2352, 'st...","{'match_id': 2999684669, 'duration': 1052, 'st...","{'match_id': 2999668332, 'duration': 2879, 'st...","{'match_id': 2999617933, 'duration': 2179, 'st...","{'match_id': 2999569127, 'duration': 1792, 'st..."


# Mongo stuff

In [8]:
client = MongoClient('mongodb://localhost:27017/')
db = client['dota']

In [ ]:
all_ = pd.DataFrame(list(db.matches_pgroup_format.find({})))

In [54]:
hero_pairs = np.array(list((itertools.combinations(heroes["id"].values, r = 2))))

np.hstack(np.array([hero_pairs ,[db for _ in range(hero_pairs.shape[0])]]))

ValueError: could not broadcast input array from shape (7021,2) into shape (7021)

In [7]:
from functools import partial

if __name__ == '__main__':
    
    list_ = heroes["id"].values[:5]
    
    
    with Pool(4) as p:

        r = tqdm.tqdm(p.map(query_methods.get_pair_win_rate
                                , itertools.combinations(list_, r = 2)))

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused

In [59]:
r

[{'Hero_1': 1, 'Hero_2': 2, 'Win_rate': 0.5071743724741389},
 {'Hero_1': 1, 'Hero_2': 3, 'Win_rate': 0.5061493157803568},
 {'Hero_1': 1, 'Hero_2': 4, 'Win_rate': 0.509805834715582},
 {'Hero_1': 1, 'Hero_2': 5, 'Win_rate': 0.515604337476858},
 {'Hero_1': 2, 'Hero_2': 3, 'Win_rate': 0.49872665534804755},
 {'Hero_1': 2, 'Hero_2': 4, 'Win_rate': 0.5107954816262332},
 {'Hero_1': 2, 'Hero_2': 5, 'Win_rate': 0.5115400755350399},
 {'Hero_1': 3, 'Hero_2': 4, 'Win_rate': 0.5012366034624897},
 {'Hero_1': 3, 'Hero_2': 5, 'Win_rate': 0.5048640558742828},
 {'Hero_1': 4, 'Hero_2': 5, 'Win_rate': 0.5112797459479523}]

In [84]:
list_ = heroes["id"].to_numpy()#[:5]
list_ = list(itertools.combinations(list_, r = 2))
len(list_)

7021

In [ ]:
np.array([0,1,2,3,4])

In [10]:
from functools import partial

if __name__ == '__main__':
    
    hero_id_list = heroes["id"].values#[:5]
    combs = list(itertools.combinations(hero_id_list, r = 2))
    
    with Pool(4) as p:

        r = list(tqdm.tqdm(p.imap(query_methods.get_pair_win_rate_new, combs), total  = len(combs)))
        


  3%|▎         | 190/7021 [25:52<13:01:30,  6.86s/it]Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-10:
Process ForkPoolWorker-9:


KeyboardInterrupt: 

In [ ]:
pairs

In [8]:
from functools import partial

if __name__ == '__main__':
    
    with Pool(4) as p:

        r = tqdm(p.starmap(query_methods.get_pair_win_rate_new
                                , itertools.combinations(heroes["id"].values
                                                         , r = 2)))

TypeError: get_pair_win_rate_new() takes 1 positional argument but 2 were given

In [60]:
pd.DataFrame(r)

,Hero_1,Hero_2,Win_rate
0,1,2,0.507174
1,1,3,0.506149
2,1,4,0.509806
3,1,5,0.515604
4,2,3,0.498727
5,2,4,0.510795
6,2,5,0.511540
7,3,4,0.501237
8,3,5,0.504864
9,4,5,0.511280


In [9]:
data = 

ValueError: Trailing data